# Excess Mortality in Europe and Why (part 2)
_Author: Justin Garza_

**Date:** See below  
  
**Description:**  
This notebook explores excess mortality across Europe, analyzing statistical trends and investigating potential causes through data visualization and interpretation.  

**Content Warning:**    
If you find discussions of death or its underlying factors distressing, please proceed with caution or consider whether this content is right for you.  

In [1]:
from datetime import datetime
from IPython.display import display
from IPython.display import Markdown as MD
current_date = datetime.now().strftime('%Y-%m-%d')
version = datetime.now().strftime('%Y%m%d.%H%M')
display(MD(f"**Date:** {current_date}"))
display(MD(f"**version:** {version}"))

**Date:** 2025-01-03

**version:** 20250103.1914

## SetUp
just like part 1

In [2]:
# this code to will import all the things i need for this notebook

import os
import re
import math

import numpy as np
import pandas as pd

import random
from collections import Counter


# for the notebook rendering 
from IPython.display import display, HTML, FileLink, Markdown
from IPython.display import FileLink as FL
from IPython.display import Markdown as MD


# Graphs and Charts
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import seaborn as sns
import plotly.express as px
# use to export plotly graphs
import plotly.io as pio 

#misc
from scipy.stats import spearmanr, kendalltau
import pycountry

# pandas Settings/Options
pd.set_option("display.max_rows", None) 
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 9000)
pd.set_option('max_colwidth', 400)

# colormap 
heatmapCM = sns.color_palette('Spectral_r', as_cmap=True)


## directories 
DIR = os.getcwd()
print(f'{DIR=}')

DataDIR = os.path.join(DIR,'data')
OutDIR = os.path.join(DIR,'docs')

if not os.path.exists(DataDIR):
    print('***DATA FOLDER IS MISSING***')

if not os.path.exists(OutDIR):
    os.makedirs(OutDIR)

DIR='c:\\Users\\JGarza\\GitHub\\Excess_Mortality_And_Vaccines_In_Europe'


## Helping Functions


In [3]:
def df_column_uniquify(df):
    '''
    renames columns that are the same
    '''
    df_columns = df.columns
    new_columns = []
    for item in df_columns:
        counter = 0
        newitem = item
        while newitem in new_columns:
            counter += 1
            newitem = "{}_{}".format(item, counter)
        new_columns.append(newitem)
    df.columns = new_columns
    return df

In [4]:
def abbr_to_isoalpha3(abbr):
    """
    Convert a European country ISO Alpha-2 code to ISO Alpha-3 code.

    Parameters:
        abbreviation (str): ISO Alpha-2 country code (e.g., 'DE' for Germany).

    Returns:
        str: ISO Alpha-3 country code (e.g., 'DEU'), or None if not found.
    """
    try:
        country = pycountry.countries.get(alpha_2=abbr.upper())
        if country:
            return country.alpha_3
        else:
            return None
    except KeyError:
        return None

In [5]:
# this if for converting between the abbreviation andand the names of the countries

country_dict = {
    "BE": "Belgium",
    "BG": "Bulgaria",
    "CZ": "Czechia",
    "DK": "Denmark",
    "DE": "Germany",
    "EE": "Estonia",
    "IE": "Ireland",
    "EL": "Greece",
    "ES": "Spain",
    "FR": "France",
    "HR": "Croatia",
    "IT": "Italy",
    "CY": "Cyprus",
    "LV": "Latvia",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "HU": "Hungary",
    "MT": "Malta",
    "NL": "Netherlands",
    "AT": "Austria",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "SI": "Slovenia",
    "SK": "Slovakia",
    "FI": "Finland",
    "SE": "Sweden",
    "IS": "Iceland",
    "LI": "Liechtenstein",
    "NO": "Norway",
    "CH": "Switzerland",
    "UK": "United Kingdom",
    "ME": "Montenegro",
    "GE": "Georgia",
    "AL": "Albania",
    "RS": "Serbia",
    "AD": "Andorra",
    "AM": "Armenia",
}

def abbr_to_name(abbreviation):
    return country_dict.get(abbreviation.upper(), "???")

def name_to_abbr(name):
    try:
        reverse_dict = {v: k for k, v in country_dict.items()}
        return reverse_dict.get(name.title(), "???")
    except:
        return "???"

# testing
print( abbr_to_name("BE") )  # Output: 'Belgium'
print( name_to_abbr("Belgium") )  # Output: 'BE'

Belgium
BE


## import data from part 1
### Variables
- `ddr2023`: Death data ranked by deaths in 2023.
- `ddr2024`: Death data ranked by deaths in 2024.
- `vdr2023`: Vaccination data ranked by the cumulative sum for 2023.
- `vdravg`: Vaccination data ranked by the average cumulative sum for 2020, 2021, 2022, and 2023.


In [6]:
ddr2023 = pd.read_csv(os.path.join(OutDIR,'ddr2023.csv'))
ddr2024 = pd.read_csv(os.path.join(OutDIR,'ddr2024.csv'))
vdr2023 = pd.read_csv(os.path.join(OutDIR,'vdr2023.csv'))
vdravg = pd.read_csv(os.path.join(OutDIR,'vdravg.csv'))

In [7]:
display(ddr2023.head(5))
display(ddr2024.head(5))
display(vdr2023.head(5))
display(vdravg.head(5))

,abbr,name,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,MT,Malta,1.004157,0.961827,1.033935,1.067663,1.071152,1.184449,1.203738,1.226999,1.169429,1.195514
1,CY,Cyprus,1.016240,0.943163,1.040284,1.003756,1.076467,1.129533,1.249761,1.247856,1.157833,1.095931
2,FI,Finland,0.983845,1.009937,1.006529,1.023471,1.011123,1.038642,1.083859,1.163166,1.150723,1.071371
3,IS,Iceland,0.975717,1.026691,0.998060,1.006559,1.012375,1.029265,1.045927,1.203398,1.147478,1.163664
4,NL,Netherlands,0.990752,0.999534,1.009892,1.031217,1.021614,1.133312,1.147735,1.140822,1.138502,1.149227


,abbr,name,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,MT,Malta,1.004157,0.961827,1.033935,1.067663,1.071152,1.184449,1.203738,1.226999,1.169429,1.195514
1,IS,Iceland,0.975717,1.026691,0.998060,1.006559,1.012375,1.029265,1.045927,1.203398,1.147478,1.163664
2,NL,Netherlands,0.990752,0.999534,1.009892,1.031217,1.021614,1.133312,1.147735,1.140822,1.138502,1.149227
3,LI,Liechtenstein,0.984876,1.050506,0.964909,1.062178,1.011598,1.240639,1.038833,1.085522,1.027161,1.127824
4,IE,Ireland,0.994335,1.004840,1.000934,1.020105,1.008454,1.028312,1.105698,1.135294,1.132809,1.113159


,abbr,name,2020,2021,2022,2023
0,FI,Finland,0.003119,123.689548,324.096693,272.896203
1,PT,Portugal,0.008074,94.765752,242.524514,206.590389
2,IT,Italy,0.004416,87.617659,233.812064,185.254382
3,SE,Sweden,0.002223,80.134686,207.824977,169.837101
4,FR,France,0.000154,85.965138,222.847236,123.812258


,abbr,name,avg2020_2023
0,FI,Finland,180.171391
1,PT,Portugal,135.972182
2,IT,Italy,126.672130
3,SE,Sweden,114.449747
4,FR,France,108.156197


## Cause of Death Data
In this section, we import and clean the cause of death data to prepare it for analysis.

### Source
- [Eurostat Database]](https://ec.europa.eu/eurostat/databrowser/view/hlth_cd_aro__custom_14850625/default/table?lang=en)

### Downloading Instructions
1. Visit the [source link](https://ec.europa.eu/eurostat/databrowser/view/hlth_cd_aro__custom_14850625/default/table?lang=en).
2. Click "Download" and select the spreadsheet format.
3. Confirm the download to obtain the data.

### Description
This is the country, Cause of Death, and the count of each death between 2015 and 2023*.  
* note: much of 2023 is not reported

### variables 
- `cod` = cause of death data 
- `cod20dd102023` = list of the top 20 cause of deaths with countries in the top 10 death data ranked by deaths in 2023


In [8]:
# getting the data
cod = pd.read_excel(os.path.join(DataDIR,"hlth_cd_aro__custom_14850625_page_spreadsheet.xlsx"),sheet_name="Sheet 1")

# remove the headers
cod = cod.iloc[9::]

# drop the bad columns
for c in cod.columns:
    if pd.isnull(cod.at[9,c]):
        cod = cod.drop(columns=[c])

# rename time columns
for c in cod.columns:
    name = cod.at[9,c]
    cod = cod.rename(columns={c: name})

# make the duplicate column names unique
cod = df_column_uniquify(cod)

# # rename the first two columns
cod = cod.rename(columns={'TIME': 'name'})
cod = cod.rename(columns={'TIME_1':'cod'})

# drop, replace, reset index,
cod = cod.drop([9,10])
cod = cod.replace(to_replace=':', value=None)
cod = cod.reset_index(drop=True)

display(cod.head(10))


c:\Users\JGarza\pythons\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,name,cod,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,European Union - 27 countries (from 2020),Total,4618660,4536667,4660352,4693334,4660270,5189007,5298536,None,None
1,European Union - 27 countries (from 2020),Certain infectious and parasitic diseases (A00-B99),81735,74272,77707,77359,77128,73199,75560,None,None
2,European Union - 27 countries (from 2020),Tuberculosis,4098,3809,3648,3597,3382,3041,2943,None,None
3,European Union - 27 countries (from 2020),Viral hepatitis and sequelae of viral hepatitis,6956,6356,5857,4861,4396,4150,3587,None,None
4,European Union - 27 countries (from 2020),Chronic viral hepatitis B and C,1127,4814,1020,4273,3830,3529,3028,None,None
5,European Union - 27 countries (from 2020),Human immunodeficiency virus [HIV] disease,3234,2737,2438,2366,2173,2027,1891,None,None
6,European Union - 27 countries (from 2020),Other infectious and parasitic diseases (remainder of A00-B99),67284,61174,65666,66119,66788,64033,67197,None,None
7,European Union - 27 countries (from 2020),Neoplasms,1200230,1213401,1212135,1216939,1218306,1215992,1193515,None,None
8,European Union - 27 countries (from 2020),"Malignant neoplasm of lip, oral cavity, pharynx",27968,28456,28285,28573,28094,27845,27571,None,None
9,European Union - 27 countries (from 2020),"Malignant neoplasm of colon, rectosigmoid junction, rectum, anus and anal canal",137921,139889,137718,137468,137428,135701,132364,None,None


In [9]:
# lets get the abbr

cod['abbr'] = cod['name'].apply(name_to_abbr)

cod.loc[cod.name.str.contains('28 countries') == True,'abbr'] = 'Eur28'
cod.loc[cod.name.str.contains('27 countries') == True,'abbr'] = 'Eur27'

display(cod.tail(250).head(10))
# display(cod.head(10))


,name,cod,2015,2016,2017,2018,2019,2020,2021,2022,2023,abbr
1975,Norway,Intentional self-harm,598,627,603,681,666,650,657,None,None,NO
1976,Norway,Accidental poisoning by and exposure to noxious substances,324,348,298,331,323,376,322,None,None,NO
1977,Norway,Assault,25,36,30,24,31,36,31,None,None,NO
1978,Norway,Event of undetermined intent,7,5,5,6,8,12,12,None,None,NO
1979,Norway,Other external causes of morbidity and mortality (remainder of V01-Y89),2,1,0,2,5,7,6,None,None,NO
1980,Switzerland,Total,68279,65533,67431,67621,68424,76853,71848,75199,None,CH
1981,Switzerland,Certain infectious and parasitic diseases (A00-B99),849,765,825,823,826,851,912,1001,None,CH
1982,Switzerland,Tuberculosis,17,25,19,25,9,13,15,12,None,CH
1983,Switzerland,Viral hepatitis and sequelae of viral hepatitis,81,61,50,13,17,52,42,33,None,CH
1984,Switzerland,Chronic viral hepatitis B and C,None,None,0,9,11,6,7,11,None,CH


In [10]:
# reformat the year

cod =  pd.melt(cod, id_vars=['name', 'abbr','cod'], var_name='year', value_name='deaths')
cod.year = pd.to_numeric(cod.year)
cod.deaths = pd.to_numeric(cod.deaths)
display(cod.head(10))

,name,abbr,cod,year,deaths
0,European Union - 27 countries (from 2020),Eur27,Total,2015,4618660.0
1,European Union - 27 countries (from 2020),Eur27,Certain infectious and parasitic diseases (A00-B99),2015,81735.0
2,European Union - 27 countries (from 2020),Eur27,Tuberculosis,2015,4098.0
3,European Union - 27 countries (from 2020),Eur27,Viral hepatitis and sequelae of viral hepatitis,2015,6956.0
4,European Union - 27 countries (from 2020),Eur27,Chronic viral hepatitis B and C,2015,1127.0
5,European Union - 27 countries (from 2020),Eur27,Human immunodeficiency virus [HIV] disease,2015,3234.0
6,European Union - 27 countries (from 2020),Eur27,Other infectious and parasitic diseases (remainder of A00-B99),2015,67284.0
7,European Union - 27 countries (from 2020),Eur27,Neoplasms,2015,1200230.0
8,European Union - 27 countries (from 2020),Eur27,"Malignant neoplasm of lip, oral cavity, pharynx",2015,27968.0
9,European Union - 27 countries (from 2020),Eur27,"Malignant neoplasm of colon, rectosigmoid junction, rectum, anus and anal canal",2015,137921.0


In [11]:
display(cod.head(10))

,name,abbr,cod,year,deaths
0,European Union - 27 countries (from 2020),Eur27,Total,2015,4618660.0
1,European Union - 27 countries (from 2020),Eur27,Certain infectious and parasitic diseases (A00-B99),2015,81735.0
2,European Union - 27 countries (from 2020),Eur27,Tuberculosis,2015,4098.0
3,European Union - 27 countries (from 2020),Eur27,Viral hepatitis and sequelae of viral hepatitis,2015,6956.0
4,European Union - 27 countries (from 2020),Eur27,Chronic viral hepatitis B and C,2015,1127.0
5,European Union - 27 countries (from 2020),Eur27,Human immunodeficiency virus [HIV] disease,2015,3234.0
6,European Union - 27 countries (from 2020),Eur27,Other infectious and parasitic diseases (remainder of A00-B99),2015,67284.0
7,European Union - 27 countries (from 2020),Eur27,Neoplasms,2015,1200230.0
8,European Union - 27 countries (from 2020),Eur27,"Malignant neoplasm of lip, oral cavity, pharynx",2015,27968.0
9,European Union - 27 countries (from 2020),Eur27,"Malignant neoplasm of colon, rectosigmoid junction, rectum, anus and anal canal",2015,137921.0


In [12]:
# here we normalize the cause of death data

cod['deaths_norm'] = None
cod.reset_index(drop=True)

# print(cod.columns)

## baseline years are 2015,2016,2017 before the pandemic 
blyears = cod[cod.year.isin([2015,2016,2017])]
grouped = blyears.groupby(['name','abbr','cod'])

temp = grouped['deaths'].agg(
    baseline='mean',
).reset_index()

for _, row in temp.iterrows():
    try:
        cod.loc[(cod.abbr == row['abbr']) & (cod.cod == row['cod']) & (cod.name == row['name']), 'deaths_norm'] = cod[(cod.abbr == row['abbr']) & (cod.cod == row['cod']) & (cod.name == row['name'])].deaths / row['baseline']
    except Exception as e:
        # can't caluate them all 
        print(f"Error: {e} | {row=}")

cod.deaths_norm = pd.to_numeric(cod.deaths_norm)


### viewing some data


In [13]:
display(cod.head(10))

,name,abbr,cod,year,deaths,deaths_norm
0,European Union - 27 countries (from 2020),Eur27,Total,2015,4618660.0,1.002917
1,European Union - 27 countries (from 2020),Eur27,Certain infectious and parasitic diseases (A00-B99),2015,81735.0,1.049167
2,European Union - 27 countries (from 2020),Eur27,Tuberculosis,2015,4098.0,1.063955
3,European Union - 27 countries (from 2020),Eur27,Viral hepatitis and sequelae of viral hepatitis,2015,6956.0,1.088633
4,European Union - 27 countries (from 2020),Eur27,Chronic viral hepatitis B and C,2015,1127.0,0.485706
5,European Union - 27 countries (from 2020),Eur27,Human immunodeficiency virus [HIV] disease,2015,3234.0,1.153764
6,European Union - 27 countries (from 2020),Eur27,Other infectious and parasitic diseases (remainder of A00-B99),2015,67284.0,1.039810
7,European Union - 27 countries (from 2020),Eur27,Neoplasms,2015,1200230.0,0.993084
8,European Union - 27 countries (from 2020),Eur27,"Malignant neoplasm of lip, oral cavity, pharynx",2015,27968.0,0.990497
9,European Union - 27 countries (from 2020),Eur27,"Malignant neoplasm of colon, rectosigmoid junction, rectum, anus and anal canal",2015,137921.0,0.995752


In [14]:
# print some random ones with the colors

abbrs = cod.abbr.drop_duplicates().to_list()
random.shuffle(abbrs)

cods = cod.cod.drop_duplicates().to_list()
random.shuffle(cods)

for a in abbrs[0:5]:
    for c in cods[0:1]:
        try:
            temp = cod[(cod.abbr == a) & (cod.cod == c) ]
            temp = pd.pivot_table(
                data =  temp,
                values='deaths_norm',
                index = ['abbr','name','cod'],
                columns='year',
                aggfunc='mean'
            )
            temp.columns.name = 'index' 
            temp = temp.reset_index()
            display(temp.style.background_gradient(cmap=heatmapCM,axis=1))
        except Exception as e:
            pass
            print(f"Error: {e} | a={a} | c={c}")


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,SK,Slovakia,Other diseases of the circulatory system (remainder of I00-I99),0.887673,0.951406,1.160921,1.087648,0.865539,0.867829,0.919349,0.945681,1.387228


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
0,PL,Poland,Other diseases of the circulatory system (remainder of I00-I99),0.956924,0.985943,1.057133,1.071189,0.973471,1.119992,1.025963,0.760243


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AT,Austria,Other diseases of the circulatory system (remainder of I00-I99),1.053585,1.005074,0.941341,0.958128,0.899943,0.925550,0.988429,0.960689,0.968513


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021
0,BE,Belgium,Other diseases of the circulatory system (remainder of I00-I99),1.044900,0.988636,0.966463,0.953714,0.901053,0.890798,0.881098


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
0,LU,Luxembourg,Other diseases of the circulatory system (remainder of I00-I99),0.996593,0.945486,1.057922,0.833049,0.879046,0.965928,0.945486,0.945486


### notes
we don't have enough 2023 and 2024 data 

In [27]:

cod_counter = Counter()
name_counter = Counter()

temp = pd.pivot_table(
        data =  cod,
        values='deaths_norm',
        index = ['abbr','name','cod'],
        columns='year',
        aggfunc='mean'
    )
temp.columns.name = 'index' 
temp = temp.reset_index()
for y in [2024,2023,2022,2021,2020]:
    try:
        temp = temp.sort_values(by=y,ascending=False)
        print(f'sorted by {y}')
        break
    except:
        pass

top_causes = temp.cod.head(1000).to_list()
cod_counter.update(top_causes)

top_name = temp.name.head(1000).to_list()
name_counter.update(top_name)

display(temp.head(100).style.background_gradient(cmap=heatmapCM,axis=1))

sorted by 2023


c:\Users\JGarza\pythons\Python312\Lib\site-packages\pandas\io\formats\style.py:3809: RuntimeWarning: invalid value encountered in scalar subtract
  rng = smax - smin


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022,2023
1305,LI,Liechtenstein,Chronic liver disease (excluding alcoholic and toxic liver disease),nan,nan,nan,nan,inf,inf,inf,inf,inf
1307,LI,Liechtenstein,Dementia,nan,1.500000,0.500000,1.500000,0.500000,8.500000,6.000000,7.500000,9.000000
1318,LI,Liechtenstein,Ill-defined and unknown causes of mortality,1.200000,1.500000,0.300000,1.800000,2.400000,2.700000,1.200000,1.500000,5.700000
168,AT,Austria,Other mental and behavioural disorders (remainder of F00-F99),0.794118,1.014706,1.191176,1.191176,2.338235,2.250000,3.352941,3.441176,4.014706
2090,SK,Slovakia,Non-malignant neoplasms (benign and uncertain),0.886640,0.607287,1.506073,1.336032,2.064777,0.838057,1.748988,2.502024,3.595142
142,AT,Austria,Influenza (including swine flu),0.717557,0.557252,1.725191,3.480916,2.374046,2.305344,0.045802,2.633588,3.442748
464,CZ,Czechia,"Other symptoms, signs and abnormal clinical and laboratory findings (remainder of R00-R99)",0.904192,0.868263,1.227545,1.485030,2.365269,3.065868,3.113772,2.856287,3.293413
726,ES,Spain,Event of undetermined intent,1.071429,0.785714,1.142857,1.761905,2.190476,0.833333,1.452381,1.880952,3.095238
1316,LI,Liechtenstein,Hodgkin disease and lymphomas,nan,nan,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,3.000000
431,CZ,Czechia,Event of undetermined intent,0.882353,0.995475,1.122172,1.506787,1.927602,2.031674,2.683258,2.918552,2.678733


In [28]:
html_table = f"""
<h>Top 20 Causes of Death in the top 1000 causes of death (see above)</h>
<table>
    <tr>
        <th>Item</th>
        <th>Count</th>
        <th>bar</th>
    </tr>
    {''.join(f'<tr><td>{i[0]}</td><td>{i[1]}</td><td>{"#" * i[1]}</td></tr>' for i in cod_counter.most_common()[0:20])}
</table>
"""

display(HTML(html_table))

Item,Count,bar
Asthma and status asthmaticus,18,##################
"Accidents (V01-X59, Y85, Y86)",18,##################
Acute myocardial infarction including subsequent myocardial infarction,18,##################
Chronic liver disease (excluding alcoholic and toxic liver disease),17,#################
Dementia,17,#################
Ill-defined and unknown causes of mortality,17,#################
Other mental and behavioural disorders (remainder of F00-F99),17,#################
Non-malignant neoplasms (benign and uncertain),17,#################
Influenza (including swine flu),17,#################
"Other symptoms, signs and abnormal clinical and laboratory findings (remainder of R00-R99)",17,#################


In [31]:
ddnames = ddr2023.name.head(100).to_list()

html_table = f"""
<h>Top 20 Countries (abbr) in the top 1000 causes of death (see above)</h>
<table>
    <tr>
        <th>Item</th>
        <th>Count</th>
        <th>bar</th>
        <th>in ddnames list</th>
    </tr>
    {''.join(f'<tr><td>{i[0]}</td><td>{i[1]}</td><td>{"#" * i[1]}</td><td>{"Y" if i[0] in ddnames else "N"}</td></tr>' for i in name_counter.most_common(20))}
</table>
"""

display(HTML(html_table))

Item,Count,bar,in ddnames list
Austria,59,###########################################################,Y
Slovakia,59,###########################################################,Y
Czechia,59,###########################################################,Y
Spain,59,###########################################################,Y
Metropolitan France,59,###########################################################,N
Türkiye,59,###########################################################,N
Belgium,59,###########################################################,Y
Bulgaria,59,###########################################################,Y
Switzerland,59,###########################################################,Y
Cyprus,59,###########################################################,Y


# lets combine it with the data from part 1

### top 10 causes of deaths for top 10 countries with the most deaths in 2023

In [19]:
abbrs = ddr2023.head(10).abbr.to_list()

counter = Counter()

print(abbrs)

for a in abbrs:
    try:
        temp = cod[(cod.abbr == a)]
        temp = pd.pivot_table(
            data =  temp,
            values='deaths_norm',
            index = ['abbr','name','cod'],
            columns='year',
            aggfunc='mean'
        )
        temp.columns.name = 'index' 
        temp = temp.reset_index()
        for y in [2024,2023,2022,2021,2020]:
            try:
                temp = temp.sort_values(by=y,ascending=False)
                print(f'sorted by {y}')
                break
            except:
                pass

        top_causes = temp.cod.head(20).to_list()
        counter.update(top_causes)
        
        display(temp.head(10).style.background_gradient(cmap=heatmapCM,axis=1))
    except Exception as e:
        pass
        print(f"Error: {e} | a={a} | c={c}")


cod20dd102023 = []
for i in counter.most_common()[0:20]:
    cod20dd102023.append(i[0])

html_table = f"""
<h>Top 20 Causes of Death in the countries with the top deaths in 2023</h>
<table>
    <tr>
        <th>Item</th>
        <th>Count</th>
        <th>bar</th>
    </tr>
    {''.join(f'<tr><td>{i[0]}</td><td>{i[1]}</td><td>{"#" * i[1]}</td></tr>' for i in counter.most_common()[0:20])}
</table>
"""

display(HTML(html_table))


['MT', 'CY', 'FI', 'IS', 'NL', 'IE', 'DE', 'DK', 'AT', 'LU']
sorted by 2021


c:\Users\JGarza\pythons\Python312\Lib\site-packages\pandas\io\formats\style.py:3809: RuntimeWarning: invalid value encountered in scalar subtract
  rng = smax - smin


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021
18,MT,Malta,Event of undetermined intent,nan,nan,nan,nan,inf,inf,inf
23,MT,Malta,Ill-defined and unknown causes of mortality,1.875000,0.375000,0.750000,3.750000,5.625000,4.125000,3.375000
43,MT,Malta,Other external causes of morbidity and mortality (remainder of V01-Y89),1.000000,1.000000,1.000000,3.000000,2.000000,5.000000,3.000000
45,MT,Malta,Other infectious and parasitic diseases (remainder of A00-B99),0.708333,1.041667,1.250000,2.000000,1.833333,2.708333,2.666667
42,MT,Malta,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.825000,0.900000,1.275000,0.975000,1.575000,1.500000,2.175000
7,MT,Malta,Certain infectious and parasitic diseases (A00-B99),0.750000,1.173913,1.076087,1.630435,1.597826,2.184783,2.152174
12,MT,Malta,Dementia,0.846316,0.966316,1.187368,1.610526,1.446316,2.040000,2.103158
50,MT,Malta,Other mental and behavioural disorders (remainder of F00-F99),2.000000,0.500000,0.500000,1.500000,1.500000,0.500000,2.000000
20,MT,Malta,Falls,0.788321,1.226277,0.985401,0.919708,1.138686,1.160584,1.686131
27,MT,Malta,Leukaemia,0.929577,0.676056,1.394366,1.140845,1.014085,1.521127,1.521127


sorted by 2022


c:\Users\JGarza\pythons\Python312\Lib\site-packages\pandas\io\formats\style.py:3809: RuntimeWarning: invalid value encountered in scalar subtract
  rng = smax - smin


...

sorted by 2023


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022,2023
50,AT,Austria,Other mental and behavioural disorders (remainder of F00-F99),0.794118,1.014706,1.191176,1.191176,2.338235,2.250000,3.352941,3.441176,4.014706
24,AT,Austria,Influenza (including swine flu),0.717557,0.557252,1.725191,3.480916,2.374046,2.305344,0.045802,2.633588,3.442748
1,AT,Austria,Accidental poisoning by and exposure to noxious substances,1.043478,0.847826,1.108696,0.815217,1.402174,1.108696,2.543478,2.576087,2.673913
23,AT,Austria,Ill-defined and unknown causes of mortality,1.178571,0.730519,1.090909,1.051948,1.081169,2.668831,2.386364,2.571429,2.610390
12,AT,Austria,Dementia,0.675152,0.902424,1.422424,1.613939,1.612727,1.490909,1.644848,1.780606,2.007273
15,AT,Austria,Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism,1.012678,0.950872,1.036450,1.364501,1.412044,0.998415,1.492868,1.673534,1.954041
9,AT,Austria,Chronic liver disease (excluding alcoholic and toxic liver disease),nan,2.000000,0.000000,1.847690,1.972534,1.940075,1.867665,1.987516,1.950062
43,AT,Austria,Other external causes of morbidity and mortality (remainder of V01-Y89),1.070876,0.838918,1.090206,1.171392,0.931701,1.055412,1.453608,1.623711,1.786082
45,AT,Austria,Other infectious and parasitic diseases (remainder of A00-B99),0.808273,1.044718,1.147010,1.287870,1.370039,1.205702,1.447177,1.351593,1.713807
39,AT,Austria,Other diseases of the musculoskeletal system and connective tissue (remainder of M00-M99),0.816406,1.089844,1.093750,1.238281,1.230469,1.125000,1.179688,1.152344,1.542969


sorted by 2022


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
4,LU,Luxembourg,Assault,1.500000,1.250000,0.250000,0.750000,1.000000,0.750000,1.250000,2.750000
23,LU,Luxembourg,Ill-defined and unknown causes of mortality,0.968750,0.968750,1.062500,1.031250,2.031250,2.218750,2.468750,2.625000
1,LU,Luxembourg,Accidental poisoning by and exposure to noxious substances,1.826087,0.391304,0.782609,1.304348,1.434783,2.347826,2.478261,2.086957
24,LU,Luxembourg,Influenza (including swine flu),1.846154,0.692308,0.461538,3.230769,2.307692,1.615385,0.000000,1.846154
45,LU,Luxembourg,Other infectious and parasitic diseases (remainder of A00-B99),0.895522,0.716418,1.388060,0.940299,0.940299,1.119403,1.074627,1.671642
38,LU,Luxembourg,Other diseases of the genitourinary system (remainder of N00-N99),0.937500,0.562500,1.500000,1.375000,1.312500,1.750000,1.625000,1.625000
7,LU,Luxembourg,Certain infectious and parasitic diseases (A00-B99),0.907895,0.697368,1.394737,0.894737,0.881579,1.092105,1.013158,1.552632
9,LU,Luxembourg,Chronic liver disease (excluding alcoholic and toxic liver disease),nan,2.000000,0.000000,1.902439,1.609756,1.902439,1.512195,1.512195
42,LU,Luxembourg,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.785235,0.926174,1.288591,0.825503,0.986577,0.966443,1.026846,1.449664
53,LU,Luxembourg,"Rheumatoid arthritis and arthrosis (M05-M06,M15-M19)",1.384615,0.923077,0.692308,0.923077,1.153846,0.923077,0.461538,1.384615


Item,Count,bar
Other mental and behavioural disorders (remainder of F00-F99),10,##########
"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",9,#########
Other diseases of the nervous system and the sense organs (remainder of G00-H95),9,#########
Ill-defined and unknown causes of mortality,8,########
Other diseases of the genitourinary system (remainder of N00-N99),8,########
Chronic liver disease (excluding alcoholic and toxic liver disease),8,########
Event of undetermined intent,7,#######
Dementia,7,#######
Accidental poisoning by and exposure to noxious substances,7,#######
Other diseases of the musculoskeletal system and connective tissue (remainder of M00-M99),7,#######


### top 10 causes of deaths for top 10 countries with the most vaccines in 2023

In [20]:
abbrs = vdr2023.head(10).abbr.to_list()

counter = Counter()

print(abbrs)

for a in abbrs:
    try:
        temp = cod[(cod.abbr == a)]
        temp = pd.pivot_table(
            data =  temp,
            values='deaths_norm',
            index = ['abbr','name','cod'],
            columns='year',
            aggfunc='mean'
        )
        temp.columns.name = 'index' 
        temp = temp.reset_index()
        for y in [2024,2023,2022,2021,2020]:
            try:
                temp = temp.sort_values(by=y,ascending=False)
                print(f'sorted by {y}')
                break
            except:
                pass

        top_causes = temp.cod.head(20).to_list()
        counter.update(top_causes)
        
        display(temp.head(10).style.background_gradient(cmap=heatmapCM,axis=1))
    except Exception as e:
        pass
        print(f"Error: {e} | a={a} | c={c}")


html_table = f"""
<h>Top 20 Causes of Death in the countries with the top vaccines in 2023</h>
<table>
    <tr>
        <th>Item</th>
        <th>Count</th>
        <th>bar</th>
        <th>in last list</th>
    </tr>
    {''.join(f'<tr><td>{i[0]}</td><td>{i[1]}</td><td>{"#" * i[1]}</td><td>{"Y" if i[0] in cod20dd102023 else "N"}</td></tr>' for i in counter.most_common(20))}
</table>
"""

display(HTML(html_table))

['FI', 'PT', 'IT', 'SE', 'FR', 'LT', 'PL', 'DK', 'IE', 'NL']
sorted by 2022


c:\Users\JGarza\pythons\Python312\Lib\site-packages\pandas\io\formats\style.py:3809: RuntimeWarning: invalid value encountered in scalar subtract
  rng = smax - smin


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
9,FI,Finland,Chronic liver disease (excluding alcoholic and toxic liver disease),nan,nan,nan,inf,inf,inf,inf,inf
11,FI,Finland,Chronic viral hepatitis B and C,nan,nan,nan,inf,inf,inf,inf,inf
23,FI,Finland,Ill-defined and unknown causes of mortality,1.464883,0.675585,0.859532,0.735786,0.682274,0.789298,1.535117,2.046823
17,FI,Finland,"Drug dependence, toxicomania (F11-F16, F18-F19)",0.579545,1.431818,0.988636,1.602273,1.363636,1.738636,1.840909,1.738636
36,FI,Finland,Other diseases of the circulatory system (remainder of I00-I99),0.949102,1.001744,1.049153,1.095639,1.214777,1.300975,1.339764,1.535557
42,FI,Finland,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.856354,1.088398,1.055249,1.110497,1.149171,1.215470,1.226519,1.497238
50,FI,Finland,Other mental and behavioural disorders (remainder of F00-F99),0.987578,1.024845,0.987578,1.229814,1.217391,1.198758,1.409938,1.490683
12,FI,Finland,Dementia,0.998815,1.005927,0.995258,1.101439,1.083658,1.144623,1.305165,1.429128
40,FI,Finland,Other diseases of the nervous system and the sense organs (remainder of G00-H95),0.967711,0.992403,1.039886,1.081671,1.171890,1.238367,1.246914,1.360874
13,FI,Finland,Diabetes mellitus,0.910006,1.021397,1.068597,1.017621,1.125236,1.261171,1.270610,1.346130


sorted by 2021


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021
17,PT,Portugal,"Drug dependence, toxicomania (F11-F16, F18-F19)",1.434783,0.391304,1.173913,1.043478,1.565217,1.695652,2.347826
50,PT,Portugal,Other mental and behavioural disorders (remainder of F00-F99),0.917197,1.136943,0.945860,1.337580,2.025478,2.149682,2.207006
1,PT,Portugal,Accidental poisoning by and exposure to noxious substances,0.864629,0.917031,1.218341,1.401747,1.414847,1.296943,2.148472
23,PT,Portugal,Ill-defined and unknown causes of mortality,1.019554,0.976008,1.004439,1.153791,1.454295,1.755518,1.835053
12,PT,Portugal,Dementia,0.887734,1.004329,1.107937,1.337374,1.544589,1.751804,1.662915
35,PT,Portugal,"Other accidents (W20-W64, W75-X39, X50-X59, Y86)",0.769384,1.017325,1.213292,1.176654,0.605794,1.011360,1.583073
45,PT,Portugal,Other infectious and parasitic diseases (remainder of A00-B99),0.930394,0.995789,1.073817,1.050037,0.995046,1.028486,1.405251
42,PT,Portugal,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",1.068902,0.974587,0.956510,0.989521,0.877129,1.107414,1.393503
2,PT,Portugal,"Accidents (V01-X59, Y85, Y86)",0.892690,0.983984,1.123327,1.093811,0.892690,1.057087,1.302483
43,PT,Portugal,Other external causes of morbidity and mortality (remainder of V01-Y89),1.018310,1.073239,0.908451,0.954930,1.212676,1.276056,1.297183


...

index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
17,IE,Ireland,"Drug dependence, toxicomania (F11-F16, F18-F19)",2.000000,nan,0.000000,2.000000,0.000000,10.000000,8.000000,32.000000
23,IE,Ireland,Ill-defined and unknown causes of mortality,1.325581,0.732558,0.941860,1.186047,1.534884,2.825581,4.430233,4.186047
43,IE,Ireland,Other external causes of morbidity and mortality (remainder of V01-Y89),1.000000,1.857143,0.142857,0.714286,1.571429,1.142857,1.571429,4.142857
35,IE,Ireland,"Other accidents (W20-W64, W75-X39, X50-X59, Y86)",0.915789,0.934737,1.149474,1.471579,1.768421,2.065263,2.349474,2.482105
9,IE,Ireland,Chronic liver disease (excluding alcoholic and toxic liver disease),1.445344,1.554656,0.000000,2.040486,2.016194,1.991903,2.574899,2.477733
51,IE,Ireland,"Other symptoms, signs and abnormal clinical and laboratory findings (remainder of R00-R99)",0.829493,0.981567,1.188940,1.147465,1.534562,1.935484,1.811060,2.211982
50,IE,Ireland,Other mental and behavioural disorders (remainder of F00-F99),0.914286,1.000000,1.085714,1.285714,1.200000,1.028571,1.771429,1.942857
39,IE,Ireland,Other diseases of the musculoskeletal system and connective tissue (remainder of M00-M99),0.906178,0.851259,1.242563,1.187643,1.112128,1.324943,1.263158,1.709382
42,IE,Ireland,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.982487,0.992995,1.024518,1.329247,1.171629,1.444834,1.528897,1.670753
38,IE,Ireland,Other diseases of the genitourinary system (remainder of N00-N99),0.936842,1.052632,1.010526,1.063158,1.126316,1.321053,1.210526,1.478947


sorted by 2022


c:\Users\JGarza\pythons\Python312\Lib\site-packages\pandas\io\formats\style.py:3809: RuntimeWarning: invalid value encountered in scalar subtract
  rng = smax - smin


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
9,NL,Netherlands,Chronic liver disease (excluding alcoholic and toxic liver disease),nan,nan,nan,inf,inf,inf,inf,inf
11,NL,Netherlands,Chronic viral hepatitis B and C,nan,nan,nan,inf,inf,inf,inf,inf
23,NL,Netherlands,Ill-defined and unknown causes of mortality,0.793319,1.035491,1.171190,1.172582,1.299930,3.081420,3.815588,4.358386
17,NL,Netherlands,"Drug dependence, toxicomania (F11-F16, F18-F19)",0.828358,1.119403,1.052239,0.962687,1.432836,1.500000,1.835821,2.037313
18,NL,Netherlands,Event of undetermined intent,1.236641,0.801527,0.961832,0.870229,1.145038,1.007634,1.259542,1.763359
20,NL,Netherlands,Falls,0.887891,1.032518,1.079591,1.264168,1.294518,1.432951,1.470115,1.686900
24,NL,Netherlands,Influenza (including swine flu),0.928808,0.851821,1.219371,3.009934,1.343543,0.735099,0.052152,1.504967
2,NL,Netherlands,"Accidents (V01-X59, Y85, Y86)",0.932499,1.017589,1.049913,1.189866,1.195981,1.255562,1.283867,1.498078
19,NL,Netherlands,External causes of morbidity and mortality (V01-Y89),0.949282,1.007179,1.043539,1.131461,1.134375,1.177449,1.205828,1.371537
1,NL,Netherlands,Accidental poisoning by and exposure to noxious substances,0.861751,1.013825,1.124424,1.059908,1.105991,1.225806,1.253456,1.350230


Item,Count,bar,in last list
"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",10,##########,Y
Chronic liver disease (excluding alcoholic and toxic liver disease),9,#########,Y
"Drug dependence, toxicomania (F11-F16, F18-F19)",9,#########,N
Other mental and behavioural disorders (remainder of F00-F99),9,#########,Y
Other diseases of the nervous system and the sense organs (remainder of G00-H95),9,#########,Y
Ill-defined and unknown causes of mortality,8,########,Y
Other diseases of the genitourinary system (remainder of N00-N99),8,########,Y
Other diseases of the musculoskeletal system and connective tissue (remainder of M00-M99),7,#######,Y
"Other accidents (W20-W64, W75-X39, X50-X59, Y86)",7,#######,N
Other infectious and parasitic diseases (remainder of A00-B99),7,#######,Y


### top 10 causes of deaths for top 10 countries with the least deaths in 2023

In [21]:
abbrs = vdr2023.tail(10).abbr.to_list()

counter = Counter()

print(abbrs)

for a in abbrs:
    try:
        temp = cod[(cod.abbr == a)]
        temp = pd.pivot_table(
            data =  temp,
            values='deaths_norm',
            index = ['abbr','name','cod'],
            columns='year',
            aggfunc='mean'
        )
        temp.columns.name = 'index' 
        temp = temp.reset_index()
        for y in [2024,2023,2022,2021,2020]:
            try:
                temp = temp.sort_values(by=y,ascending=False)
                print(f'sorted by {y}')
                break
            except:
                pass

        top_causes = temp.cod.head(20).to_list()
        counter.update(top_causes)
        
        display(temp.head(10).style.background_gradient(cmap=heatmapCM,axis=1))
    except Exception as e:
        pass
        print(f"Error: {e} | a={a} | c={c}")


html_table = f"""
<h>Top 20 Causes of Death in the countries with the top vaccines in 2023</h>
<table>
    <tr>
        <th>Item</th>
        <th>Count</th>
        <th>bar</th>
        <th>in last list</th>
    </tr>
    {''.join(f'<tr><td>{i[0]}</td><td>{i[1]}</td><td>{"#" * i[1]}</td><td>{"Y" if i[0] in cod20dd102023 else "N"}</td></tr>' for i in counter.most_common(20))}
</table>
"""

display(HTML(html_table))

['EE', 'HU', 'AT', 'SI', 'HR', 'LV', 'SK', 'IS', 'RO', 'BG']
sorted by 2022


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
43,EE,Estonia,Other external causes of morbidity and mortality (remainder of V01-Y89),1.000000,1.500000,0.500000,2.500000,4.500000,6.750000,5.750000,4.750000
42,EE,Estonia,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.904110,1.027397,1.068493,0.657534,1.849315,2.178082,3.452055,3.657534
11,EE,Estonia,Chronic viral hepatitis B and C,nan,2.000000,0.000000,3.714286,1.142857,2.285714,2.000000,2.857143
12,EE,Estonia,Dementia,0.547445,0.875912,1.576642,1.729927,2.562044,2.321168,2.583942,2.583942
13,EE,Estonia,Diabetes mellitus,0.800384,1.053743,1.145873,1.324376,1.410749,1.923225,2.136276,2.452975
45,EE,Estonia,Other infectious and parasitic diseases (remainder of A00-B99),0.763314,0.852071,1.384615,1.384615,1.739645,2.023669,2.183432,2.236686
9,EE,Estonia,Chronic liver disease (excluding alcoholic and toxic liver disease),nan,2.000000,0.000000,2.289474,1.736842,1.815789,2.236842,2.131579
38,EE,Estonia,Other diseases of the genitourinary system (remainder of N00-N99),1.128440,0.743119,1.128440,1.403670,2.036697,2.366972,2.146789,2.091743
53,EE,Estonia,"Rheumatoid arthritis and arthrosis (M05-M06,M15-M19)",1.066667,1.066667,0.866667,1.000000,1.800000,1.000000,1.600000,2.066667
20,EE,Estonia,Falls,1.111446,0.876506,1.012048,0.885542,1.048193,1.644578,1.635542,1.942771


sorted by 2022


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
38,HU,Hungary,Other diseases of the genitourinary system (remainder of N00-N99),1.034007,1.003676,0.962316,1.006434,1.202206,1.466912,1.715074,2.023897
23,HU,Hungary,Ill-defined and unknown causes of mortality,0.949843,0.921630,1.128527,1.316614,1.636364,2.040752,1.937304,2.021944
51,HU,Hungary,"Other symptoms, signs and abnormal clinical and laboratory findings (remainder of R00-R99)",0.745033,1.142384,1.112583,1.072848,1.251656,2.155629,1.827815,1.639073
42,HU,Hungary,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.972022,0.972022,1.055957,1.072202,1.245487,1.629964,1.778881,1.619134
9,HU,Hungary,Chronic liver disease (excluding alcoholic and toxic liver disease),1.500000,1.500000,0.000000,1.453349,1.543062,1.578947,1.589713,1.604067
22,HU,Hungary,Human immunodeficiency virus [HIV] disease,1.178571,0.964286,0.857143,0.964286,1.500000,0.642857,1.071429,1.392857
34,HU,Hungary,Non-malignant neoplasms (benign and uncertain),0.851395,1.015558,1.133047,1.068670,1.007511,1.332618,1.242489,1.305258
15,HU,Hungary,Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism,0.939689,1.044747,1.015564,1.056420,1.132296,1.132296,1.143969,1.295720
18,HU,Hungary,Event of undetermined intent,1.147059,0.970588,0.882353,0.916667,0.941176,1.029412,1.102941,1.279412
45,HU,Hungary,Other infectious and parasitic diseases (remainder of A00-B99),1.064578,0.970588,0.964834,0.991688,0.941816,1.116368,1.396419,1.269821


sorted by 2023


...

sorted by 2021


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021
17,RO,Romania,"Drug dependence, toxicomania (F11-F16, F18-F19)",0.857143,1.714286,0.428571,1.285714,3.428571,2.571429,4.285714
20,RO,Romania,Falls,0.940820,1.034683,1.024497,1.130730,1.319913,1.242784,2.512491
51,RO,Romania,Pneumonia,0.987941,0.956995,1.055063,1.241900,1.327619,1.725846,2.146593
45,RO,Romania,Other infectious and parasitic diseases (remainder of A00-B99),0.792081,1.042034,1.165885,1.370801,1.700694,1.798086,1.900544
41,RO,Romania,Other diseases of the respiratory system (remainder of J00-J99),0.952059,1.017387,1.030554,1.281735,1.466070,1.870189,1.861074
42,RO,Romania,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.858696,1.201087,0.940217,1.168478,1.293478,1.532609,1.793478
0,RO,Romania,Accidental drowning and submersion,1.052141,0.982986,0.964874,1.050494,0.956641,0.852909,1.679473
13,RO,Romania,Diabetes mellitus,0.970099,1.037763,0.992138,1.070628,1.106973,1.529579,1.654079
50,RO,Romania,Other mental and behavioural disorders (remainder of F00-F99),0.958333,1.010417,1.031250,1.312500,1.010417,1.364583,1.625000
7,RO,Romania,Certain infectious and parasitic diseases (A00-B99),0.893048,1.024916,1.082037,1.198237,1.376782,1.382983,1.443695


sorted by 2022


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
24,BG,Bulgaria,Influenza (including swine flu),0.360000,1.560000,1.080000,0.960000,1.560000,6.480000,6.600000,2.880000
18,BG,Bulgaria,Event of undetermined intent,1.049661,0.934537,1.015801,1.049661,1.550790,2.465011,2.458239,2.857788
11,BG,Bulgaria,Chronic viral hepatitis B and C,nan,2.000000,0.000000,2.666667,2.833333,3.333333,5.666667,2.166667
9,BG,Bulgaria,Chronic liver disease (excluding alcoholic and toxic liver disease),nan,2.000000,0.000000,1.908169,2.069407,1.986119,2.085424,2.084357
23,BG,Bulgaria,Ill-defined and unknown causes of mortality,0.935599,1.010733,1.053667,0.967800,0.962433,2.903399,1.695886,2.055456
52,BG,Bulgaria,Pneumonia,0.969735,0.980898,1.049367,0.963036,1.092533,2.212602,2.644257,1.964773
58,BG,Bulgaria,Viral hepatitis and sequelae of viral hepatitis,0.847059,0.952941,1.200000,1.305882,1.376471,1.270588,2.047059,1.341176
43,BG,Bulgaria,Other external causes of morbidity and mortality (remainder of V01-Y89),1.036364,1.072727,0.890909,0.781818,1.200000,0.600000,1.381818,1.309091
39,BG,Bulgaria,Other diseases of the musculoskeletal system and connective tissue (remainder of M00-M99),1.186047,0.802326,1.011628,0.976744,1.151163,1.081395,1.325581,1.255814
37,BG,Bulgaria,Other diseases of the digestive system (remainder of K00-K93),0.982722,1.012647,1.004631,1.038831,1.049519,1.113644,1.164945,1.217314


Item,Count,bar,in last list
Other diseases of the genitourinary system (remainder of N00-N99),8,########,Y
Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism,8,########,N
Ill-defined and unknown causes of mortality,8,########,Y
Dementia,7,#######,Y
Chronic liver disease (excluding alcoholic and toxic liver disease),7,#######,Y
Other mental and behavioural disorders (remainder of F00-F99),7,#######,Y
Non-malignant neoplasms (benign and uncertain),7,#######,Y
Other diseases of the nervous system and the sense organs (remainder of G00-H95),7,#######,Y
Diabetes mellitus,6,######,N
Other infectious and parasitic diseases (remainder of A00-B99),6,######,Y
